# OLG model under complete market
In the following code, 
- use the backward induction method.
- do not consider the uncertainty of life-span


In [ ]:
using BenchmarkTools
using Plots

In [2]:
type Model
    # primitive parameter
    beta::Float64 #subjective discount factor
    gamma::Float64 # weight for cons
    sigma::Float64 # relative risk aversion
    delta::Float64 #depriciation rate
    alpha::Float64 # capital share
    
    # discretize asset space
    agrid::Array{Float64,1}
end

In [120]:
function OLG(m::Model)
    # primitive params
    const beta = m.beta
    const gamma = m.gamma
    const sigma = m.sigma
    const delta = m.delta
    const alpha = m.alpha
    
    # demog data
    const J = 66 # life-span
    const JR = 46 # age of retirement
    const tR = J -JR + 1 # length of retirement
    const tW = JR - 1 # length of working life
    const n  = 0.011 # pop.growth
    
    e = zeros(tW)
    e[1:11]  = linspace(0.6, 1.0, 11)
    e[11:21] = linspace(1.0, 1.08, 11) 
    e[21:31] = linspace(1.08, 1.12, 11)
    e[31:41] = linspace(1.12, 1.06, 11)
    e[41:45] = linspace(1.06, 1.02, 5)
    
    # social security tax rate
    const tau = 0.11 
    # if you want to consider the case without ss, tau = 0.0
    
    # discritized asset
    const agrid = m.agrid
    const na =size(agrid,1)
    
    # measure of each generation 
    mass = ones(J)
    for j in 2:J
        mass[j] = mass[j-1]/(1+n)
    end
    mass = mass/sum(mass) # normalized
   
    
    
    # initial guess for capital and labor
    K0 = 3.0
    L0 = 0.3
    
    # iteration settings
    const penalty = -99999999999999999999999.9
    const tolk = 1e-3
    const toll = 1e-3
    err_k = 10.0
    err_l = 10.0
    const maxiter = 100
    iter = 0
    val_temp = -99999999999.9
    
    
    # initialization
    v = zeros(na,J) # true value function
    v0 = zeros(na,na,J) # temp value function
    a_index::Array{Int64,2}= zeros(na,J)
    pol_a = zeros(na,J) # policy for capital
    pol_l = zeros(na,tW) # policy for labor
    cons = zeros(na,1) # last period consumption
    c    = zeros(na,na,J) # each period consumption
    util = zeros(na,na,J)
    u(c) = (c^(1.0-sigma))/(1.0-sigma)
    
    k_gen = zeros(J) # capital supply for each generation
    l_gen = zeros(tW) # labor supply for each generation
    r0 = 0.0
    w0 = 0.0
    b  = 0.0
    while tolk < err_k || toll < err_l
        
        # factor price
        r0 = alpha*(K0^(alpha-1.0))*(L0^(1.0-alpha)) -delta
        w0 = (1.0 - alpha)*(K0^(alpha))*(L0^(-alpha))
        
        # social security benefit
        b = (tau*w0*L0)/(sum(mass[JR:end]))
        
        ## backward induction
        
        # retired people
        
        # last period util
        for i in 1 : na
            cons[i] = (1+r0)*agrid[i] + b
            v[i,J]  = (cons[i]^(1.0-sigma))/(1-sigma)
        end
        
        # in this case we compute utility for the retired
        choice_id ::Int64 = 1
        # retired
        for j in J-1 : -1 : tW+1
            for i in 1:na
                for aa in 1 : na
                    c[i,aa,j] = (1+r0)*agrid[i] + b - agrid[aa]
                    util[i,aa,j] = u(c[i,aa,j])
                    if c[i,aa,j] < 0.0
                        util[i,aa,j] = penalty 
                    end
                    v0[i,aa,j] = util[i,aa,j] +beta*v[aa,j+1]
                    
                    if v0[i,aa,j] >= val_temp
                        val_temp = copy(v0[i,aa,j])
                        choice_id = aa
                        #pol_a[i,j] = agrid[aa]
                    end     
                end
                v[i,j] = copy(val_temp)
                a_index[i,j] = choice_id
                pol_a[i,j] = agrid[choice_id]
                
            end
        end
        
        # we compute util for the working 
        for j in tW : -1 : 1
            for i in 1:na
                for aa in 1:na
                    # caluculate optimal labor supply
                    lab = 0.0
                    lab =(gamma*(1.0-tau)*e[j]*w0 -(1.0-gamma)*((1.0+r0)*agrid[i] - agrid[aa]))/((1.0-tau)*e[j]*w0)
                    # feasiblity 
                    if lab > 1.0
                        lab = 1.0
                    elseif lab < 0.0
                        lab = 0.0
                    end
                        
                    c[i,aa,j] = (1.0+r0)*agrid[i] +(1.0-tau)*w0*e[j]*lab - agrid[aa]
                    util[i,aa,j] = u(c[i,aa,j])
                    if c[i,aa,j] < 0.0
                        util[i,aa,j] = penalty 
                    end
                    
                    v0[i,aa,j] = util[i,aa] + beta*v[aa,j+1]
                    if v0[i,aa,j]>= val_temp
                        val_temp = copy(v0[i,aa,j])

                        choice_id = aa
        
                        pol_l[i,j] = copy(lab)
                    end
                        
                end
                v[i,j] = copy(val_temp)
                a_index[i,j] = choice_id
                pol_a[i,j] = agrid[choice_id]
            end
        end
        #println(a_index)
        # aggregate capital and labor
        ik0::Int64 = 1                
        ik1::Int64 =1
        for j in 1:J-1

            ik1 = a_index[ik0, j]
            
            k_gen[j] = agrid[ik1]

            if j<=tW
               l_gen[j] =pol_l[ik0, j]
            end
        
            ik1 = ik0
        end
       
        K1 = dot(k_gen , mass)
        L1 = dot(l_gen, mass[1:tW])
        
        # update
        K0 = 0.9*K0 + 0.1*K1
        L0 = 0.9*L0 + 0.1*L1
        err_k = abs(K0-K1)
        err_l = abs(L0-L1)
        iter += 1
        println("the number of iteration is $iter")
        println("deviation-capital is $err_k")
        println("deviation-labor is $err_l")
        if iter >= maxiter
            println("the model cannot convege in $maxiter of iterations")
            break
        end
    end
            
                
    
   return(v,pol_a,pol_l,K0,L0)     
end
    

OLG (generic function with 1 method)

In [121]:
beta = 0.95 #subjective discount factor
gamma = 0.4 # weight on cons
sigma = 2.0 # relative risk aversion
delta = 0.1 #depriciation rate
alpha = 0.36 # capital share



amin = 0.01
amax = 10.0
na   = 250

model = Model(beta, gamma, sigma, delta, alpha, linspace(amin, amax, na))

Model(0.95, 0.4, 2.0, 0.1, 0.36, [0.01, 0.0501205, 0.090241, 0.130361, 0.170482, 0.210602, 0.250723, 0.290843, 0.330964, 0.371084  …  9.63892, 9.67904, 9.71916, 9.75928, 9.7994, 9.83952, 9.87964, 9.91976, 9.95988, 10.0])

In [122]:
v,pol_a,pol_l,K,L = OLG(model)

the number of iteration is 1
deviation-capital is 5.0810107856104905
deviation-labor is 0.39138499728619974
the number of iteration is 2
deviation-capital is 3.199194596503252
deviation-labor is 0.35224649755757975
the number of iteration is 3
deviation-capital is 2.879275136852927
deviation-labor is 0.3170218478018218
the number of iteration is 4
deviation-capital is 2.5913476231676342
deviation-labor is 0.28531966302163964
the number of iteration is 5
deviation-capital is 2.3322128608508708
deviation-labor is 0.25678769671947566
the number of iteration is 6
deviation-capital is 2.0989915747657837
deviation-labor is 0.23110892704752806
the number of iteration is 7
deviation-capital is 1.8890924172892056
deviation-labor is 0.2079980343427752
the number of iteration is 8
deviation-capital is 1.700183175560285
deviation-labor is 0.18719823090849763
the number of iteration is 9
deviation-capital is 1.5301648580042564
deviation-labor is 0.1684784078176479
the number of iteration is 10
devi

the number of iteration is 76
deviation-capital is 0.34236622395726884
deviation-labor is 0.0001448079422107229
the number of iteration is 77
deviation-capital is 0.30812960156154195
deviation-labor is 0.0001303271479895507
the number of iteration is 78
deviation-capital is 0.27731664140538775
deviation-labor is 0.00011729443319052901
the number of iteration is 79
deviation-capital is 0.24958497726484902
deviation-labor is 0.00010556498987146501
the number of iteration is 80
deviation-capital is 0.22462647953836412
deviation-labor is 9.50084908842852e-5
the number of iteration is 81
deviation-capital is 0.20216383158452772
deviation-labor is 8.550764179582337e-5
the number of iteration is 82
deviation-capital is 0.18194744842607496
deviation-labor is 7.695687761621883e-5
the number of iteration is 83
deviation-capital is 0.16375270358346747
deviation-labor is 6.926118985450813e-5
the number of iteration is 84
deviation-capital is 0.14737743322512073
deviation-labor is 6.23350708689463e

([-0.0400992 -0.0400992 … -0.0400992 -9.37856; -0.0400992 -0.0400992 … -0.0400992 -4.24754; … ; -0.0400992 -0.0400992 … -0.0400992 -0.0312012; -0.0400992 -0.0400992 … -0.0400992 -0.0310763], [0.01 0.01 … 0.01 0.0; 0.01 0.01 … 0.01 0.0; … ; 0.01 0.01 … 0.01 0.0; 0.01 0.01 … 0.01 0.0], [0.0 1.0 … 1.0 1.0; 0.0 1.0 … 1.0 1.0; … ; 0.0 0.554256 … 0.495849 0.496788; 0.0 0.525937 … 0.478252 0.479019], 0.03720542709006143, 0.7348606683924097)